In [ ]:
# author: Jan Tschada
# SPDX-License-Identifer: Apache-2.0

In [ ]:
from arcgis.gis import GIS
from arcgis.features import FeatureSet
from datetime import datetime
from georapid.client import GeoRapidClient
from georapid.factory import EnvironmentClientFactory
from geourban import services, types

In [ ]:
class UrbanSimulation(object):
    
    def __init__(self):
        self._client = EnvironmentClientFactory.create_client_with_host('geourban.p.rapidapi.com')
        self._simulations = services.simulations(self._client)
    
    def find_simulation(self, region_name):
        for simulation in self._simulations:
            if region_name == simulation['name']:
                return simulation
        
        raise ValueError(f'No traffic simulation for "{region_name}" exists!')
    
    def query_car_hotspots(self, region_code, simulation_date):
        vehicle_type = types.VehicleType.CAR
        grid_type = types.GridType.AGENT
        traffic_grid = services.top(self._client, region_code, simulation_date, vehicle_type, grid_type)
        return FeatureSet.from_geojson(traffic_grid).sdf

In [ ]:
urban_sim = UrbanSimulation()
bonn_simulation = urban_sim.find_simulation('Bonn, Kreisfreie Stadt')
region_code = bonn_simulation['region']
simulation_date = datetime.fromisoformat(bonn_simulation['date']).date()
car_speed_spatial_df = urban_sim.query_car_hotspots(region_code, simulation_date)
gis = GIS()
bonn_map = gis.map('Bonn, Germany', zoomlevel=13)
car_speed_spatial_df.spatial.plot(bonn_map, renderer_type='s', colors='#E80000', alpha=0.3)
bonn_map